In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import BatchNormalization
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image 
import seaborn as sns
import os
import re
import glob
import cv2
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn import preprocessing
import tqdm
from numpy import loadtxt
import shutil

from scipy.stats import uniform, randint

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from tensorflow.keras.applications.vgg16 import preprocess_input

In [2]:
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)', text)]

In [3]:
def return_count(x):
    name_wel = []
    for i in x:
        name_wel.append(i.split('_features')[0])

    z = list(set(name_wel))
    r = list(range(len(z)))

    num = []
    for iz in range(len(z)):
        count = 0
        for i in x:
            if z[iz] in i:
                count += 1
        num.append(count)
    return list(zip(z, r, num))

In [4]:
def creat_label(y, indirizo):
    labels = []
    for ix, _ in enumerate(y):
        labels.append([[y[ix][0],y[ix][1]]] * y[ix][2])
    
    ler = [i for sub in labels for i in sub ]

    df = pd.DataFrame(ler, columns = ['Type', 'Category'])
    df = df.sort_values(by=['Type'])
    df.to_csv(indirizo, sep=',',index=False)
    return 'Done'

In [5]:
def resize():
    rescaled = []

    for i in x_orig_1:

        scale_percent = 140 # percent of original size
        width = int(i.shape[1] / (scale_percent / 100))
        height = int(i.shape[0] / (scale_percent / 100))
        dim = (width, height)
        resized = cv2.resize(i, dim, interpolation = cv2.INTER_LANCZOS4)
        rescaled.append(resized)

    x_orig = np.reshape( rescaled, (len( rescaled), resized.shape[1], resized.shape[1], 3))

    return x_orig

In [6]:
p = '/home/jovyan/Saved_Models/CNN_same_fields_view_ADR_HRH.h5'
m4 = load_model(p)

In [7]:
m4.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Model)                (None, 8, 8, 256)         1735488   
_________________________________________________________________
batch_normalization (BatchNo (None, 8, 8, 256)         1024      
_________________________________________________________________
global_average_pooling2d (Gl (None, 256)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                16448     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6

In [8]:
base_model = Model(inputs=m4.input, outputs=m4.get_layer('dense').output)


In [11]:
path =  '/home/jovyan/DATA_MASTER_PROJECT/Check_DIFF_T60_T_97/2_class_ADRR_HRH/VALIDATION/'

name_well = []
for di,dr,f in os.walk(path):
    for i in f:
        name_well.append(i.split('_id')[0])

wells = list(set(name_well))
wells

for w in tqdm.tqdm(wells):

    time = []
    for dirname, dirnames,filenames in os.walk(path):

        for filename in sorted(filenames, key = natural_keys):
            if w in filename: #PAY ATTENTION ID THE IMAGE IS A TIFF OR PNG IMAGE #########
                time.append(filename)

    data_id = {}
    n_id = []
    w_n = []

    for i in time:
        t = i.split('_id_')[1].split('time_')[0]
        f = i.split('_id_')[0].split('time_')[0]
        n_id.append(t)
        w_n.append(f)

    id_cell = set(n_id)


    for ix, i in enumerate(sorted(id_cell, key = natural_keys)):

        id_name = []
        dict_1 = {}

        for t in time:
            if 'id_{}'.format(i) in t:
                id_name.append(t)

        d = {'id':id_name}
        data = pd.DataFrame(d)

        dict_1[ix]=data 
        data_id.update(dict_1) 

    delete = [i for i, j in data_id.items() if len(j) < 35] # 9 or the length of time span you are traning on 
    for i in delete : del data_id[i]

    len_id = [i for i, j in data_id.items()]


    for le in len_id:    
        image_list = []

        e = pd.DataFrame(data_id[le])

        coords = e.values.tolist()
        id_cell = []
        for i in coords:
            for j in i:
                id_cell.append(j)


        for idc in id_cell:
            for filename in glob.glob(path + idc ): 
                im=cv2.imread(filename)

                image_list.append(im)


        x_orig_1 = np.reshape(image_list, (len(image_list), 90, 90, 3))
        x_orig = resize()
        
        x_orig = preprocess_input(x_orig)
        output = base_model.predict(x_orig)
        np.save('/home/jovyan/DATA_MASTER_PROJECT/LSTM/LST_vs_CNN/ADR_HRH/features_validation/features_well_{}_id_{}.npy'.format(w_n[0], le), output)

100%|██████████| 72/72 [00:55<00:00,  1.30it/s]


In [12]:
controll = ['B3', 'D5','D6','F10']
adr = ['C6','F11', 'D11', 'G4', 'F2', 'G10', 'G5', 'B10', 'G3', 'B8']
hrh = ['E4', 'G6', 'G8', 'D10', 'E7', 'B7', 'E10', 'B11', 'D8', 'B2']

In [25]:
directory = '/home/jovyan/DATA_MASTER_PROJECT/LSTM/LST_vs_CNN/ADR_HRH/features_validation/'

In [26]:
for i in os.listdir(directory):
    for c in controll:
        if c in i:
            os.rename(directory + i, directory + 'controll_' + i)
            
            
    for a in adr:
        if a in i:
            os.rename(directory + i, directory + 'adr_' + i)
            
    for h in hrh:
        if h in i:
            os.rename(directory + i, directory + 'hrh_' + i)
            

In [27]:
feat = []
for dire, dir_name, filnames in os.walk(directory):
    for f_name in filnames:
        feat.append(f_name)

In [28]:
#train, test = train_test_split(feat, test_size=0.2, random_state=99 )
#len(train), len(test)

In [29]:
y = return_count(feat)
y

[('hrh', 0, 244), ('adr', 1, 175)]

In [ ]:
# Training

In [ ]:
#pa = '/home/jovyan/DATA_MASTER_PROJECT/LSTM/LSTM_Final_Data/'
#new_path = '/home/jovyan/Feature_LSTM_Train'
#os.mkdir(new_path)

#for i in train:
 #   shutil.copy(pa + i, new_path+ '/' +i)

# Test

In [ ]:
#new_path = '/home/jovyan/Feature_LSTM_Test'
#os.mkdir(new_path)

#for i in test:
 #   shutil.copy(pa+i, new_path+ '/' +i)

# Create Labels

In [30]:
indi = '/home/jovyan/DATA_MASTER_PROJECT/LSTM/LST_vs_CNN/ADR_HRH/'
tr = return_count(feat)
creat_label(tr, indi + '/' + 'lstm_validation.csv')

'Done'

In [ ]:
#te = return_count(train)
#creat_label(te, indi + '/' + 'lstm_test.csv')

In [ ]:
dire = '/home/jovyan/DATA_MASTER_PROJECT/LSTM/LSTM_Controll_ADR/features_train/'

In [ ]:
sh = np.load(dire+feat[120])

In [ ]:
plt.plot(sh)
plt.show